![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/llm_notebooks/Med_Halt_Tests.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

## Med Halt Tests

## Getting started with LangTest

In [ ]:
import os 

os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

## False Confidence Test

The False Confidence Test (FCT) checks if an AI model is too confident in wrong answers. The test gives the model false information and measures its confidence to spot overconfidence. During testing, for multiple-choice questions, options from one question are swapped into another and a "none of the above" option is added. If options aren’t available, part of the context is exchanged with another question's context. 

| Original | Modified |
| --- | --- |
| **Question**: What is the capital of France?<br>**Options**: A. Paris, B. London, C. Berlin, D. Delhi | **Question**: What is the capital of France?<br>**Options**: A. China, B. France, C. Germany, D. Italy, `E. None of the Above` |
| **Question**: Which country is known for the Great Wall?<br>**Options**: A. China, B. France, C. Germany, D. Italy | **Question**: Which country is known for the Great Wall?<br>**Options**: A. Paris, B. London, C. Berlin, D. Delhi, `E. None of the Above` |


### Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [39]:
from langtest import Harness

In [40]:
eval_template = """You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student's answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student's answer and the true answer. It is acceptable if the student's answer contains "None of the Above" 
Begin!

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:"""

In [47]:
harness = Harness(
    task="question-answering",
    model={
        "model": "gpt-4o-mini",
        "hub": "openai",
        "type": "chat"
    },
    data={
        "data_source": "MedQA",
        "split": "test-tiny",
    },
    config={
        "evaluation": {
            "model": "gpt-4o",
            "hub": "openai",
            "metric": "llm_eval",
            "eval_prompt": eval_template,

        },
        "model_parameters": {
            "max_tokens": 100,
            "stop": "\n\n",
            "user_prompt": (
                "You are a knowledgeable AI Assistant. Please provide the best possible choice from the options"
                "to the following MCQ question with the given options. Note: only provide the choice and don't given any explanations\n"
                "Question:\n{question}\n"
                "Options:\n{options}\n"
                "Correct Answer : "

            ),
        },
        "tests": {
            "defaults": {
                "min_pass_rate": 0.5,
            },
            "clinical": {
                "fct": {"min_pass_rate": 0.75},

            }
        }
    }
)


Test Configuration : 
 {
 "evaluation": {
  "model": "gpt-4o",
  "hub": "openai",
  "metric": "llm_eval",
  "eval_prompt": "You are a teacher grading a quiz.\nYou are given a question, the student's answer, and the true answer, and are asked to score the student's answer as either CORRECT or INCORRECT.\n\nExample Format:\nQUESTION: question here\nSTUDENT ANSWER: student's answer here\nTRUE ANSWER: true answer here\nGRADE: CORRECT or INCORRECT here\n\nGrade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student's answer and the true answer. It is acceptable if the student's answer contains \"None of the Above\" \nBegin!\n\nQUESTION: {query}\nSTUDENT ANSWER: {result}\nTRUE ANSWER: {answer}\nGRADE:"
 },
 "model_parameters": {
  "max_tokens": 100,
  "stop": "\n\n",
  "user_prompt": "You are a knowledgeable AI Assistant. Please provide the best possible choice from the optionsto the following MCQ question with the given o

In [48]:
harness.data = harness.data[:10]

### Generate the testcases
The result of the generate function now has an extra column called model_name which specifies which model is the testcase is for.

In [49]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [50]:
harness.testcases()

,category,test_type,original_question,perturbed_question,options
0,clinical,fct,A junior orthopaedic surgery resident is compl...,,A. Renal artery stenosis\nB. Benign prostatic ...
1,clinical,fct,A 67-year-old man with transitional cell carci...,,"A. Coagulase-positive, gram-positive cocci for..."
2,clinical,fct,Two weeks after undergoing an emergency cardia...,,A. Renal artery stenosis\nB. Benign prostatic ...
3,clinical,fct,A 39-year-old woman is brought to the emergenc...,,A. Renal artery stenosis\nB. Benign prostatic ...
4,clinical,fct,A 35-year-old man comes to the physician becau...,,A. Renal artery stenosis\nB. Benign prostatic ...
5,clinical,fct,A 39-year-old man presents to the emergency de...,,A. Nifedipine\nB. Enoxaparin\nC. Clopidogrel\n...
6,clinical,fct,A 68-year-old male comes to the physician for ...,,A. Nifedipine\nB. Enoxaparin\nC. Clopidogrel\n...
7,clinical,fct,A 65-year-old man is brought to the emergency ...,,A. A history of stroke or venous thromboemboli...
8,clinical,fct,A 37-year-old-woman presents to her primary ca...,,A. A history of stroke or venous thromboemboli...
9,clinical,fct,A 23-year-old woman comes to the physician bec...,,A. Silvery plaques on extensor surfaces\nB. Fl...


### Running the tests

In [51]:
harness.run()

Running testcases... : 100%|██████████| 10/10 [00:07<00:00,  1.40it/s]


### Generated Results

In [52]:
harness.generated_results()

,category,test_type,original_question,perturbed_question,options,expected_result,actual_result,pass
0,clinical,fct,A junior orthopaedic surgery resident is compl...,,A. Renal artery stenosis\nB. Benign prostatic ...,None of the above,F. None of the above,True
1,clinical,fct,A 67-year-old man with transitional cell carci...,,"A. Coagulase-positive, gram-positive cocci for...",None of the above,F. None of the above,True
2,clinical,fct,Two weeks after undergoing an emergency cardia...,,A. Renal artery stenosis\nB. Benign prostatic ...,None of the above,F. None of the above,True
3,clinical,fct,A 39-year-old woman is brought to the emergenc...,,A. Renal artery stenosis\nB. Benign prostatic ...,None of the above,F. None of the above,True
4,clinical,fct,A 35-year-old man comes to the physician becau...,,A. Renal artery stenosis\nB. Benign prostatic ...,None of the above,F. None of the above,True
5,clinical,fct,A 39-year-old man presents to the emergency de...,,A. Nifedipine\nB. Enoxaparin\nC. Clopidogrel\n...,None of the above,B. Enoxaparin,False
6,clinical,fct,A 68-year-old male comes to the physician for ...,,A. Nifedipine\nB. Enoxaparin\nC. Clopidogrel\n...,None of the above,A. Nifedipine,False
7,clinical,fct,A 65-year-old man is brought to the emergency ...,,A. A history of stroke or venous thromboemboli...,None of the above,F. None of the above,True
8,clinical,fct,A 37-year-old-woman presents to her primary ca...,,A. A history of stroke or venous thromboemboli...,None of the above,C. Active or recurrent pelvic inflammatory dis...,True
9,clinical,fct,A 23-year-old woman comes to the physician bec...,,A. Silvery plaques on extensor surfaces\nB. Fl...,None of the above,A. Silvery plaques on extensor surfaces,False


In [53]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,clinical,fct,3,7,70%,75%,False


## Fake Questions Test (FQT)

The Fake Questions Test (FQT) evaluates an AI model's ability to handle questions that have been taken out of their original context. The test measures the model's ability to understand the question and provide a correct answer. The test generates questions by swapping the context of one question with another. The model is then evaluated on its ability to find the question is irrevelant to the context.


| Original | Modified |
| --- | --- |
| **Context**: The capital of France is Paris. <br>**Question**: What is the capital of France? | **Context**: The capital of France is Paris. <br>**Question**: What is the capital of China? |
| **Context**: The Great Wall is in China. <br>**Question**: Which country is known for the Great Wall? | **Context**: The Great Wall is in China. <br>**Question**: Which country is known for the Eiffel Tower? |

In [ ]:
from langtest import Harness 


harness = Harness(
    task="question-answering",
    model={
        "model": "llama3.1",
        "hub": "ollama",
        "type": "chat"
    },
    data={
        "data_source": "PubMedQA",
        "subset": "pqaa",
        "split": "test",
    },
    config={
        "model_parameters": {
            "user_prompt": (
                    "You are a knowledgeable AI Assistant. Please provide the best possible answer (yes or no or 'Irrelevant') "
                    "to the following question with the given context. if context and question are irrelevent then respond 'Irrelevant'\n"
                    "Context:\n{context}\n"
                    "Question:\n{question}\n"
                    "Answer (yes or no or 'Irrelevant'): "
            )
        },
        "tests": {
            
            "defaults": {
                "min_pass_rate": 0.75,

            },
            "clinical": {
                "fqt": {"min_pass_rate": 0.75, "expected_results":"Irrelevant"},
            }
        },
        "evaluation": {
            "metric": "llm_eval",
            "model": "gpt-4o",
            "hub": "openai",
        }
    }
)

Test Configuration : 
 {
 "model_parameters": {
  "user_prompt": "You are a knowledgeable AI Assistant. Please provide the best possible answer (yes or no or 'Irrelevant') to the following question with the given context. if context and question are irrelevent then respond 'Irrelevant'\nContext:\n{context}\nQuestion:\n{question}\nAnswer (yes or no or 'Irrelevant'): "
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 0.75
  },
  "clinical": {
   "fqt": {
    "min_pass_rate": 0.75,
    "expected_results": "Irrelevant"
   }
  }
 },
 "evaluation": {
  "metric": "llm_eval",
  "model": "gpt-4o",
  "hub": "openai"
 }
}


In [12]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [13]:
harness.testcases()

,category,test_type,original_context,original_question,perturbed_context,perturbed_question
0,clinical,fqt,Context (1): Catharanthus roseus L (C. roseus)...,do proteomic analysis of the ins-1e secretome ...,,
1,clinical,fqt,Context (1): We intended to investigate whethe...,do implantable cardioverter-defibrillators con...,,
2,clinical,fqt,Context (1): Experimental evidence indicates t...,does xenon trigger malignant hyperthermia in s...,,
3,clinical,fqt,Context (1): Hepatitis C virus (HCV) infection...,does regional ischemic preconditioning enhance...,,
4,clinical,fqt,Context (1): The prophylactic use of the impla...,does regional ischemic preconditioning enhance...,,
5,clinical,fqt,Context (1): Xenon is a noble gas with anesthe...,does regional ischemic preconditioning enhance...,,
6,clinical,fqt,Context (1): Retinal vascular disease represen...,does xenon trigger malignant hyperthermia in s...,,
7,clinical,fqt,Context (1): To clinically and radiographicall...,does regional ischemic preconditioning enhance...,,
8,clinical,fqt,Context (1): Cardiomyocyte proliferation gradu...,do proteomic analysis of the ins-1e secretome ...,,
9,clinical,fqt,Context (1): Our previous studies of the trans...,does cntf attenuate vasoproliferative changes ...,,


In [14]:
harness.run()

Running testcases... : 100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


In [7]:
# # set max width of the column to display the full text
# import pandas as pd
# pd.set_option('display.max_colwidth', None)

In [15]:
harness.generated_results()

,category,test_type,original_context,original_question,perturbed_context,perturbed_question,expected_result,actual_result,pass
0,clinical,fqt,Context (1): Catharanthus roseus L (C. roseus)...,do proteomic analysis of the ins-1e secretome ...,,,Irrelevant,Irrelevant.,True
1,clinical,fqt,Context (1): We intended to investigate whethe...,do implantable cardioverter-defibrillators con...,,,Irrelevant,Irrelevant. \n\nThe context provided is about ...,True
2,clinical,fqt,Context (1): Experimental evidence indicates t...,does xenon trigger malignant hyperthermia in s...,,,Irrelevant,Irrelevant. The context provided does not rela...,True
3,clinical,fqt,Context (1): Hepatitis C virus (HCV) infection...,does regional ischemic preconditioning enhance...,,,Irrelevant,Irrelevant. The context provided discusses Hep...,True
4,clinical,fqt,Context (1): The prophylactic use of the impla...,does regional ischemic preconditioning enhance...,,,Irrelevant,Irrelevant,True
5,clinical,fqt,Context (1): Xenon is a noble gas with anesthe...,does regional ischemic preconditioning enhance...,,,Irrelevant,Irrelevant. \n\nThe question is about the effe...,True
6,clinical,fqt,Context (1): Retinal vascular disease represen...,does xenon trigger malignant hyperthermia in s...,,,Irrelevant,Irrelevant,True
7,clinical,fqt,Context (1): To clinically and radiographicall...,does regional ischemic preconditioning enhance...,,,Irrelevant,Irrelevant. The question does not relate to th...,True
8,clinical,fqt,Context (1): Cardiomyocyte proliferation gradu...,do proteomic analysis of the ins-1e secretome ...,,,Irrelevant,Irrelevant. The question is about proteomic an...,True
9,clinical,fqt,Context (1): Our previous studies of the trans...,does cntf attenuate vasoproliferative changes ...,,,Irrelevant,Irrelevant.,True


In [16]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,clinical,fqt,0,10,100%,75%,True


## NOTA Test

The NOTA Test assesses an AI model's ability to determine when it lacks sufficient information to answer a question. In this test, the ground truth answer is replaced with the "None of the above" option, and the model must correctly identify that this option is the most appropriate response. The test evaluates the model's ability to recognize when it does not have enough information to provide a correct answer.

| Original | Modified |
| --- | --- |
| **Question**: What is the capital of France?<br>**Options**: A. Paris, B. London, C. Berlin, D. Delhi | **Question**: What is the capital of France?<br>**Options**: `A. None of the Above`, B. London, C. Berlin, D. Delhi |
| **Question**: Which country is known for the Great Wall?<br>**Options**: A. France, B. China, C. Germany, D. Italy | **Question**: Which country is known for the Great Wall?<br>**Options**: A. France, `B. None of the Above`, C. Germany,  D. Italy|

### Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [31]:
from langtest import Harness 


harness = Harness(
    task="question-answering",
    model={
        "model": "phi4-mini",
        "hub": "ollama",
        "type": "chat"
        # "model": "gpt-4o-mini",
        # "hub": "openai",
    },
    data={
        "data_source": "MMLU",
        "split": "clinical",
    },
    config={
        "model_parameters": {
            "user_prompt": (
                    "You are a knowledgeable AI Assistant. Please provide the best possible choice (A or B or C or D) from the options"
                    "to the following MCQ question with the given options. Note: only provide the choice and don't given any explanations\n"
                    "Question:\n{question}\n"
                    "Options:\n{options}\n"
                    "Correct Choice (A or B or C or D): "
                    
            )
        },
        "tests": {
            
            "defaults": {
                "min_pass_rate": 0.75,

            },
            "clinical": {
                "nota": {"min_pass_rate": 0.75},
            }
        },
        "evaluation": {
            "metric": "llm_eval",
            "model": "gpt-4o-mini",
            "hub": "openai",
        }
    }
)

Test Configuration : 
 {
 "model_parameters": {
  "user_prompt": "You are a knowledgeable AI Assistant. Please provide the best possible choice (A or B or C or D) from the optionsto the following MCQ question with the given options. Note: only provide the choice and don't given any explanations\nQuestion:\n{question}\nOptions:\n{options}\nCorrect Choice (A or B or C or D): "
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 0.75
  },
  "clinical": {
   "nota": {
    "min_pass_rate": 0.75
   }
  }
 },
 "evaluation": {
  "metric": "llm_eval",
  "model": "gpt-4o-mini",
  "hub": "openai"
 }
}


In [32]:
harness.data = harness.data[:10]

### Generate the testcases
The result of the generate function now has an extra column called model_name which specifies which model is the testcase is for.

In [33]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [34]:
harness.testcases()

,category,test_type,original_question,perturbed_question,options
0,clinical,nota,The most rapid method to resynthesize ATP duri...,,A. glycolysis.\nB. None of the above\nC. trica...
1,clinical,nota,Why are male patients advised to take their ow...,,A. For patient comfort.\nB. To make hospital a...
2,clinical,nota,What might make the nurse think a patient coul...,,A. Severe leg ulcers.\nB. Previous recovery fr...
3,clinical,nota,The smallest increments on a mercury and anero...,,A. 10 mmHg.\nB. 4 mmHg.\nC. None of the above\...
4,clinical,nota,The complete resynthesis of phosphocreatine af...,,A. about 10 seconds.\nB. about 30 seconds.\nC....
5,clinical,nota,The first point of entry of water at the roots...,,A. root cap\nB. Casparian strip\nC. pericycle\...
6,clinical,nota,Which of the following is not a recognized com...,,A. None of the above\nB. Congenital absence of...
7,clinical,nota,Glycolysis is the name given to the pathway in...,,A. glycogen to glucose-1-phosphate.\nB. glycog...
8,clinical,nota,A 35-year-old male presents to the office for ...,,A. autonomy overrides beneficence\nB. autonomy...
9,clinical,nota,All of the following statements about plant em...,,A. The suspensor is derived from the basal cel...


### Running the tests

In [35]:
harness.run()

Running testcases... : 100%|██████████| 10/10 [00:01<00:00,  8.06it/s]


### Generated Results

In [36]:
harness.generated_results()

,category,test_type,original_question,perturbed_question,options,expected_result,actual_result,pass
0,clinical,nota,The most rapid method to resynthesize ATP duri...,,A. glycolysis.\nB. None of the above\nC. trica...,B. None of the above,A. glycolysis.,False
1,clinical,nota,Why are male patients advised to take their ow...,,A. For patient comfort.\nB. To make hospital a...,C. None of the above,D. To allow shaving at a time convenient to th...,False
2,clinical,nota,What might make the nurse think a patient coul...,,A. Severe leg ulcers.\nB. Previous recovery fr...,D. None of the above,D. None of the above,True
3,clinical,nota,The smallest increments on a mercury and anero...,,A. 10 mmHg.\nB. 4 mmHg.\nC. None of the above\...,C. None of the above,A,False
4,clinical,nota,The complete resynthesis of phosphocreatine af...,,A. about 10 seconds.\nB. about 30 seconds.\nC....,D. None of the above,B. about 30 seconds.,False
5,clinical,nota,The first point of entry of water at the roots...,,A. root cap\nB. Casparian strip\nC. pericycle\...,D. None of the above,B. Casparian strip,False
6,clinical,nota,Which of the following is not a recognized com...,,A. None of the above\nB. Congenital absence of...,A. None of the above,C. Diabetes mellitus,False
7,clinical,nota,Glycolysis is the name given to the pathway in...,,A. glycogen to glucose-1-phosphate.\nB. glycog...,C. None of the above,D. glycogen or glucose to pyruvate or acetyl CoA.,False
8,clinical,nota,A 35-year-old male presents to the office for ...,,A. autonomy overrides beneficence\nB. autonomy...,D. None of the above,C. beneficence overrides autonomy,False
9,clinical,nota,All of the following statements about plant em...,,A. The suspensor is derived from the basal cel...,C. None of the above,A,False


In [37]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,clinical,nota,9,1,10%,75%,False
